# (Dynamic) Multiple Dispatch in Julia

[Multiple dispatch](http://en.wikipedia.org/wiki/Multiple_dispatch) can be thought of as a generalization of **object-oriented** (OO) programming.

In a typical OO language like Python, an object type (class) *owns* certain *methods* (functions), and are typically called via
```
object.method(arg1, arg2)
```
Depending on the type of `object`, the runtime system will *dispatch* to different `method` definitions.

In Julia, the same call would be "spelled" differently:
```
method(object, arg1, arg2)
```
Spelled this way, you should notice something odd about OO programming: why is the *first* argument so special?

Traditional OO programming corresponds to **single dispatch**: the runtime chooses `method` based on the type of the *first* argument only.   Julia implements **multiple dispatch**: the runtime chooses `method` based on the types of *all* the arguments.

## Example: Binary mathematical operators

A classic example of the need for multiple dispatch is the case of binary math operators.  If you compute `x * y`, the definition of the `*` function depends upon *both* the arguments, not just on `x`.

Julia defines *many versions* of the `*` function:

In [1]:
methods(*)

# 115 methods for generic function "*":
*(x::Bool,y::Bool) at bool.jl:41
*{T<:Unsigned}(x::Bool,y::T<:Unsigned) at bool.jl:56
*(x::Bool,z::Complex{T<:Real}) at complex.jl:116
*{T<:Number}(x::Bool,y::T<:Number) at bool.jl:52
*(z::Complex{T<:Real},x::Bool) at complex.jl:117
*(y::Number,x::Bool) at bool.jl:58
*{T,S}(A::Union(SubArray{T,2,A<:DenseArray{T,N},I<:(Union(Range{Int64},Int64)...,)},DenseArray{T,2}),B::Union(DenseArray{S,2},SubArray{S,2,A<:DenseArray{T,N},I<:(Union(Range{Int64},Int64)...,)})) at linalg/matmul.jl:116
*(A::Union(Hermitian{T},Symmetric{T}),B::Union(Hermitian{T},Symmetric{T})) at linalg/symmetric.jl:35
*(A::Union(Hermitian{T},Symmetric{T}),B::Union(SubArray{T,2,A<:DenseArray{T,N},I<:(Union(Range{Int64},Int64)...,)},DenseArray{T,2})) at linalg/symmetric.jl:36
*(A::Union(SubArray{T,2,A<:DenseArray{T,N},I<:(Union(Range{Int64},Int64)...,)},DenseArray{T,2}),B::Union(Hermitian{T},Symmetric{T})) at linalg/symmetric.jl:37
*{T<:Union(Float64,Complex{Float32},Float32,Complex{Float64}),S<:Union(SubArray{T,2,A<:DenseArray{T,N},I<:(Union(Range{Int64},Int64)...,)},DenseArray{T,2}),UpLo,IsUnit}(A::Triangular{T<:Union(Float64,Complex{Float32},Float32,Complex{Float64}),S<:Union(SubArray{T,2,A<:DenseArray{T,N},I<:(Union(Range{Int64},Int64)...,)},DenseArray{T,2}),UpLo,IsUnit},B::Triangular{T<:Union(Float64,Complex{Float32},Float32,Complex{Float64}),S<:Union(SubArray{T,2,A<:DenseArray{T,N},I<:(Union(Range{Int64},Int64)...,)},DenseArray{T,2}),UpLo,IsUnit}) at linalg/triangular.jl:23
*(A::Tridiagonal{T},B::Triangular{T,S<:AbstractArray{T,2},UpLo,IsUnit}) at linalg/triangular.jl:206
*{TA,TB,SA<:AbstractArray{T,2},SB<:AbstractArray{T,2},UpLoA,UpLoB,IsUnitA,IsUnitB}(A::Triangular{TA,SA<:AbstractArray{T,2},UpLoA,IsUnitA},B::Triangular{TB,SB<:AbstractArray{T,2},UpLoB,IsUnitB}) at linalg/triangular.jl:209
*(Da::Diagonal{T},Db::Diagonal{T}) at linalg/diagonal.jl:53
*(A::Union(SymTridiagonal{T},Tridiagonal{T},Diagonal{T},Bidiagonal{T},Triangular{T,S<:AbstractArray{T,2},UpLo,IsUnit}),B::Union(SymTridiagonal{T},Tridiagonal{T},Diagonal{T},Bidiagonal{T},Triangular{T,S<:AbstractArray{T,2},UpLo,IsUnit})) at linalg/bidiag.jl:114
*{T<:Union(Int16,Int8,Int32)}(x::T<:Union(Int16,Int8,Int32),y::T<:Union(Int16,Int8,Int32)) at int.jl:18
*{T<:Union(Uint8,Uint16,Uint32)}(x::T<:Union(Uint8,Uint16,Uint32),y::T<:Union(Uint8,Uint16,Uint32)) at int.jl:22
*(x::Int64,y::Int64) at int.jl:47
*(x::Uint64,y::Uint64) at int.jl:48
*(x::Int128,y::Int128) at int.jl:586
*(x::Uint128,y::Uint128) at int.jl:587
*(x::Float32,y::Float32) at float.jl:123
*(x::Float64,y::Float64) at float.jl:124
*(z::Complex{T<:Real},w::Complex{T<:Real}) at complex.jl:112
*(x::Real,z::Complex{T<:Real}) at complex.jl:118
*(z::Complex{T<:Real},x::Real) at complex.jl:119
*(x::Rational{T<:Integer},y::Rational{T<:Integer}) at rational.jl:118
*(a::Float16,b::Float16) at float16.jl:132
*(x::BigInt,y::BigInt) at gmp.jl:195
*(a::BigInt,b::BigInt,c::BigInt) at gmp.jl:218
*(a::BigInt,b::BigInt,c::BigInt,d::BigInt) at gmp.jl:224
*(a::BigInt,b::BigInt,c::BigInt,d::BigInt,e::BigInt) at gmp.jl:231
*(x::BigInt,c::Union(Uint8,Uint16,Uint64,Uint32)) at gmp.jl:265
*(c::Union(Uint8,Uint16,Uint64,Uint32),x::BigInt) at gmp.jl:269
*(x::BigInt,c::Union(Int16,Int8,Int32,Int64)) at gmp.jl:271
*(c::Union(Int16,Int8,Int32,Int64),x::BigInt) at gmp.jl:275
*(x::BigFloat,y::BigFloat) at mpfr.jl:149
*(x::BigFloat,c::Union(Uint8,Uint16,Uint64,Uint32)) at mpfr.jl:156
*(c::Union(Uint8,Uint16,Uint64,Uint32),x::BigFloat) at mpfr.jl:160
*(x::BigFloat,c::Union(Int16,Int8,Int32,Int64)) at mpfr.jl:164
*(c::Union(Int16,Int8,Int32,Int64),x::BigFloat) at mpfr.jl:168
*(x::BigFloat,c::Union(Float64,Float16,Float32)) at mpfr.jl:172
*(c::Union(Float64,Float16,Float32),x::BigFloat) at mpfr.jl:176
*(x::BigFloat,c::BigInt) at mpfr.jl:180
*(c::BigInt,x::BigFloat) at mpfr.jl:184
*(a::BigFloat,b::BigFloat,c::BigFloat) at mpfr.jl:255
*(a::BigFloat,b::BigFloat,c::BigFloat,d::BigFloat) at mpfr.jl:261
*(a::BigFloat,b::BigFloat,c::BigFloat,d::BigFloa

We can add new methods to a given function at any time.  The methods don't "belong" to a particular type, and aren't part of the type's definition.

For example, string concatenation in Julia is done via `*`:

In [2]:
"hello" * "world"

"helloworld"

In [3]:
"hello" + "world"

LoadError: `+` has no method matching +(::ASCIIString, ::ASCIIString)
while loading In[3], in expression starting on line 1

But we can easily extend `+` to support a concatenation for strings, if we want:

In [4]:
import Base.+ # we must import a method to add methods (as opposed to replacing it)
+(x::String, y::String) = x * " " * y

+ (generic function with 120 methods)

In [5]:
"hello" + "world"

"hello world"

## Not the same as C++ overloading: Dynamic vs. static

This may look a lot like function overloading in languages like C++.   The difference is that C++'s overloading is **static** (= *dispatch at compile-time*), whereas Julia's overloading is dynamic (= *dispatch at run-time*), like OO polymorphism.

For example, now that we've defined `+`, we can use strings with any previously defined function that requires a `+` operation, like `sum` (summation):

In [6]:
sum(["The", "quick", "brown", "fox", "jumped", "over", "the", "lazy", "dog."])

"The quick brown fox jumped over the lazy dog."

## Type declarations are "function filters"

Type declarations are *not* required for performance — Julia automatically specializes a function on its argument types during compilation.  They act like **filters**, allowing us to specify *which functions are used when*.

Without this, in a language like Python, you sometimes have to write *manual function filters* like this example from Matplotlib's [quiver.py](https://github.com/matplotlib/matplotlib/blob/43150ef012e405de5c2fb7b779d1bffa77f7af81/lib/matplotlib/quiver.py):
```python
def _parse_args(*args):
    X, Y, U, V, C = [None] * 5
    args = list(args)
    # The use of atleast_1d allows for handling scalar arguments while also
    # keeping masked arrays
    if len(args) == 3 or len(args) == 5:
        C = np.atleast_1d(args.pop(-1))
    V = np.atleast_1d(args.pop(-1))
    U = np.atleast_1d(args.pop(-1))
    if U.ndim == 1:
        nr, nc = 1, U.shape[0]
    else:
        nr, nc = U.shape
    if len(args) == 2: # remaining after removing U,V,C
        X, Y = [np.array(a).ravel() for a in args]
        if len(X) == nc and len(Y) == nr:
            X, Y = [a.ravel() for a in np.meshgrid(X, Y)]
    else:
        indexgrid = np.meshgrid(np.arange(nc), np.arange(nr))
        X, Y = [np.ravel(a) for a in indexgrid]
    return X, Y, U, V, C
```
In Julia, you could define different methods for differing numbers of arguments, arrays vs. scalars, etcetera (all eventually calling a single lower-level function to do the work once the arguments have been transformed).